<a href="https://colab.research.google.com/github/pandian-raja/EVA4_Session11/blob/master/Session10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install albumentations


In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import numpy as np
from tqdm import tqdm
# from torch_lr_finder import LRFinder
import torch
import torch.nn as nn
import torch.nn.functional as F


class Layer(nn.Module):

    def __init__(self, in_planes, planes, resBlock=False):
        super(Layer, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = in_planes, out_channels = planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.pool = nn.MaxPool2d(2, 2)
        self.bn1 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        if resBlock:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels = planes, out_channels = planes, kernel_size=3, stride=1, padding=1, bias=False),
                nn.BatchNorm2d(planes),
                nn.ReLU(inplace=False),
                nn.Conv2d(in_channels = planes, out_channels = planes, kernel_size=3, stride=1,padding=1, bias=False),
                nn.BatchNorm2d(planes),
                nn.ReLU(inplace=False)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.pool(self.conv1(x))),inplace=False)
        r1 = self.shortcut(out)
        out = out + r1
        return out


class Session11(nn.Module):
    def __init__(self, num_classes=10):
        super(Session11, self).__init__()
        self.conv1 = nn.Conv2d(in_channels= 3, out_channels =64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = nn.Sequential(Layer(64, 128,True))
        self.layer2 = nn.Sequential(Layer(128, 256,False))
        self.layer3 = nn.Sequential(Layer(256, 512,True))
        self.linear = nn.Linear(512, num_classes,bias=False)

    # def _make_layer(self, block, planes, num_blocks, resBlock):
    #     strides = [stride] + [1]*(num_blocks-1)
    #     layers = []
    #     for stride in strides:
    #         layers.append(block(self.in_planes, planes, stride))
    #         self.in_planes = planes * block.expansion
    #     return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)),inplace=False)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.max_pool2d(input = out, kernel_size=4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        # x = out.view(-1, 10)
        return F.log_softmax(out, dim=-1)
        

# net = Session11()
# print(net)
# summary(net.cuda(), input_size=(3, 32, 32))


In [9]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay= 0.0001)
lr_finder = LRFinder(model, optimizer, criterion, device)


NameError: ignored

In [0]:

import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import albumentations as A
from torch.utils.data import DataLoader, Dataset
from albumentations.pytorch import ToTensor
from albumentations import Compose, RandomCrop,PadIfNeeded,Flip, Normalize, HorizontalFlip, Cutout, VerticalFlip, Rotate, RGBShift
from albumentations.pytorch import ToTensor

class albumCompose:
    def __init__(self):
        self.albumentations_transform = Compose({
            PadIfNeeded(min_height=40,min_width=40),
            RandomCrop(32,32),
            Flip(),
            VerticalFlip(p=0.5)
            Cutout(max_h_size=8,max_w_size=8,num_holes=1),
            Normalize((0.49139968, 0.48215841, 0.44653091), (0.24703223, 0.24348513, 0.26158784))
        })
    def __call__(self, img):
        img = np.array(img)
        img = self.albumentations_transform(image=img)['image']
        img = np.transpose(img, (2, 0, 1)).astype(np.float32)
        return torch.tensor(img, dtype=torch.float)

class albumCompose_test:
    def __init__(self):
        self.albumentations_transform = Compose({
        Normalize((0.49139968, 0.48215841, 0.44653091), (0.24703223, 0.24348513, 0.26158784))
        })
    def __call__(self, img):
        img = np.array(img)
        img = self.albumentations_transform(image=img)['image']
        img = np.transpose(img, (2, 0, 1)).astype(np.float32)
        return torch.tensor(img, dtype=torch.float)


class GetData():
    def importDataset():
        SEED = 1
        # CUDA?
        cuda = torch.cuda.is_available()
        print("CUDA Available?", cuda)
        # For reproducibility
        device = torch.device("cuda" if cuda else "cpu")
        
        torch.manual_seed(SEED)
        
        if cuda:
            torch.cuda.manual_seed(SEED)

        # train_transforms = transforms.Compose([
        #                                 transforms.RandomCrop(32, padding=4),
        #                                 transforms.RandomHorizontalFlip(),
        #                                 transforms.Cutout()
        #                                 transforms.ToTensor(),
        #                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values.
        #                                ])
        # test_transforms = transforms.Compose([
        #                                transforms.ToTensor(),
        #                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        #                                ])
        trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=albumCompose())
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=512,
                                              shuffle=True, num_workers=4)
        testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=albumCompose_test())
        testloader = torch.utils.data.DataLoader(testset, batch_size=512,
                                             shuffle=False, num_workers=4)
        classes = ('plane', 'car', 'bird', 'cat',
               'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
        return trainloader, testloader, classes, device


In [4]:
trainloader, testloader, classes, device = GetData.importDataset()

CUDA Available? True
Files already downloaded and verified
Files already downloaded and verified


In [0]:
model = Session11().to(device);

In [0]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    test_losses = 0.0
    test_acc = 0.0
    pred_wrong = []
    true_wrong = []
    image = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            _, predicted = torch.max(output.data, 1)
            preds = predicted.cpu().numpy()
            tar = target.cpu().numpy()
            preds = np.reshape(preds,(len(preds),1))
            tar = np.reshape(tar,(len(preds),1))
            for i in range(len(preds)):
                # pred.append(preds[i])
                # true.append(target[i])
                if(preds[i]!=tar[i]):
                    pred_wrong.append(preds[i])
                    true_wrong.append(tar[i])
                    image.append(data[i])
    test_loss /= len(test_loader.dataset)
    test_losses = test_loss

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc = 100. * correct / len(test_loader.dataset)
    
    return image,true_wrong,pred_wrong,test_acc,test_losses


In [0]:
criterion = nn.CrossEntropyLoss()
model = Session11().to(device);
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, nesterov=True, weight_decay= 0.0001)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, max_lr=0.009,pct_start =0.20,epochs=24, steps_per_epoch=len(trainloader))
# ,div_factor =8,final_div_factor = 8)
# scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, max_lr=0.0152480,pct_start =5/24,steps_per_epoch=len(trainloader),total_steps = 24,div_factor =8,final_div_factor = 8,epochs = 24)


In [8]:
train_losses = []
train_acc = []
test_losses = []
test_acc = []
    
for epoch in range(24):
    correct = 0
    processed = 0
    pbar = tqdm(trainloader)
    model.train()
    for batch_idx, (data, target) in enumerate(pbar):
        # get samples        
        data, target = data.to(device), target.to(device)
        # Init
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
        # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.
        # Predict
        y_pred = model(data)
        loss = F.nll_loss(y_pred, target)
        loss.backward()
        optimizer.step()
        scheduler.step()
        pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f' Epoch = {(epoch+1)}Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        # train_acc.append(100*correct/processed)

    train_losses.append(loss.item())    
    train_acc.append(100*correct/processed)
    
    
    img,true_wrong,pred_wrong,tst_acc ,tst_loss = test(model, device, testloader)
    test_losses.append(tst_loss)
    test_acc.append(tst_acc)
        

 Epoch = 1Loss=1.521882176399231 Batch_id=97 Accuracy=33.88: 100%|██████████| 98/98 [00:14<00:00,  6.65it/s]
  0%|          | 0/98 [00:00<?, ?it/s]


Test set: Average loss: 1.4494, Accuracy: 4680/10000 (46.80%)



 Epoch = 2Loss=1.2190661430358887 Batch_id=97 Accuracy=52.81: 100%|██████████| 98/98 [00:14<00:00,  6.64it/s]
  0%|          | 0/98 [00:00<?, ?it/s]


Test set: Average loss: 1.2408, Accuracy: 5543/10000 (55.43%)



 Epoch = 3Loss=1.2562021017074585 Batch_id=97 Accuracy=58.70: 100%|██████████| 98/98 [00:14<00:00,  6.66it/s]
  0%|          | 0/98 [00:00<?, ?it/s]

...


Test set: Average loss: 0.4050, Accuracy: 8621/10000 (86.21%)



 Epoch = 23Loss=0.2117154896259308 Batch_id=97 Accuracy=91.12: 100%|██████████| 98/98 [00:14<00:00,  6.63it/s]
  0%|          | 0/98 [00:00<?, ?it/s]


Test set: Average loss: 0.4009, Accuracy: 8649/10000 (86.49%)



 Epoch = 24Loss=0.2705129086971283 Batch_id=97 Accuracy=91.27: 100%|██████████| 98/98 [00:14<00:00,  6.64it/s]



Test set: Average loss: 0.4003, Accuracy: 8648/10000 (86.48%)



In [10]:
train_losses = []
train_acc = []
test_losses = []
test_acc = []
    
for epoch in range(24):
    correct = 0
    processed = 0
    pbar = tqdm(trainloader)
    model.train()
    for batch_idx, (data, target) in enumerate(pbar):
        # get samples        
        data, target = data.to(device), target.to(device)
        # Init
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
        # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.
        # Predict
        y_pred = model(data)
        loss = F.nll_loss(y_pred, target)
        loss.backward()
        optimizer.step()
        scheduler.step()
        pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        # train_acc.append(100*correct/processed)

    train_losses.append(loss.item())    
    train_acc.append(100*correct/processed)
    # scheduler.step()
    
    img,true_wrong,pred_wrong,tst_acc ,tst_loss = test(model, device, testloader)
    test_losses.append(tst_loss)
    test_acc.append(tst_acc)
        

Loss=1.500083327293396 Batch_id=97 Accuracy=35.61: 100%|██████████| 98/98 [00:14<00:00,  6.65it/s]
  0%|          | 0/98 [00:00<?, ?it/s]


Test set: Average loss: 1.3781, Accuracy: 5018/10000 (50.18%)



Loss=1.0908637046813965 Batch_id=97 Accuracy=53.61: 100%|██████████| 98/98 [00:14<00:00,  6.62it/s]
  0%|          | 0/98 [00:00<?, ?it/s]


Test set: Average loss: 1.1662, Accuracy: 5837/10000 (58.37%)



Loss=1.0783381462097168 Batch_id=97 Accuracy=60.13: 100%|██████████| 98/98 [00:14<00:00,  6.62it/s]
  0%|          | 0/98 [00:00<?, ?it/s]

...


Test set: Average loss: 0.3800, Accuracy: 8726/10000 (87.26%)



Loss=0.232192724943161 Batch_id=97 Accuracy=90.63: 100%|██████████| 98/98 [00:14<00:00,  6.59it/s]
  0%|          | 0/98 [00:00<?, ?it/s]


Test set: Average loss: 0.3763, Accuracy: 8731/10000 (87.31%)



Loss=0.28625577688217163 Batch_id=97 Accuracy=90.90: 100%|██████████| 98/98 [00:14<00:00,  6.61it/s]



Test set: Average loss: 0.3749, Accuracy: 8732/10000 (87.32%)

